In [5]:
with open('hw2/small.txt') as f:
    words_matrix = []
    lines = f.readlines()
    vocalubary = set()
    labels = []
    for l in lines:
        words = l.split()
        labels.append(int(words[0]))
        words_matrix.append(words[1:])
        for w in words[1:]:
            vocalubary.add(w)

In [ ]:
#https://stackoverflow.com/questions/41924453/pytorch-how-to-use-dataloaders-for-custom-datasets

In [2]:
words_matrix

[['I', 'have', 'an', 'apple'], ['I', 'have', 'a', 'pen'], ['apple', 'pen']]

In [6]:
vocalubary

{'I', 'a', 'an', 'apple', 'have', 'pen'}

In [7]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
le.fit(list(vocalubary))
len(list(le.classes_))

6

In [9]:
n_samples = len(words_matrix)

In [10]:
import torch
bag_of_words_matrix = torch.zeros(n_samples, len(vocalubary))

In [12]:
for lidx, line in enumerate(words_matrix):
    if lidx % 500 == 0:
        print(lidx)
    indexes = le.transform(line)
    for i in indexes:
        bag_of_words_matrix[lidx][i] = 1

0


In [13]:
bag_of_words_matrix

tensor([[1., 0., 1., 1., 1., 0.],
        [1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.]])

In [14]:
le.classes_

array(['I', 'a', 'an', 'apple', 'have', 'pen'], dtype='<U5')

In [15]:
import torch.utils.data as data_utils


In [21]:
train = data_utils.TensorDataset(bag_of_words_matrix, torch.IntTensor(labels))
train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)

In [29]:
for i in train_loader:
    print(i)

[tensor([[1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.],
        [1., 1., 0., 0., 1., 1.]]), tensor([0, 1, 1], dtype=torch.int32)]


In [57]:
import pandas as pd
s=words_matrix[0]

In [73]:
import numpy as np
dummy_train = pd.get_dummies(list(vocalubary))
dummy_test = pd.get_dummies(['ajfingfg','apple'])
dummy_test = dummy_test.reindex(columns = dummy_train.columns, fill_value=0)
torch.IntTensor(dummy_test.values.any(axis=0).astype('int'))

tensor([0, 0, 0, 1, 0, 0], dtype=torch.int32)

In [62]:
dummy_test

,ajfingfg,apple
0,1,0
1,0,1


In [44]:
def preprocess(filename):
    #read in tokens
    vocalubary = set()
    with open(filename) as f:
        words_matrix = []
        lines = f.readlines()
        labels = []
        for l in lines:
            words = l.split()
            labels.append(int(words[0]))
            words_matrix.append(words[1:])
            for w in words[1:]:
                vocalubary.add(w)
    return vocalubary, words_matrix, labels
    

In [45]:
def encode(vocalubary, words_matrix, labels):
    le=preprocessing.LabelEncoder()
    le.fit(list(vocalubary))
    bag_of_words_matrix = torch.zeros(n_samples, len(vocalubary))
    for lidx, line in enumerate(words_matrix):
        if lidx % 500 == 0:
            print(lidx)
        indexes = le.transform(line)
        for i in indexes:
            bag_of_words_matrix[lidx][i] = 1
    ds = data_utils.TensorDataset(bag_of_words_matrix, torch.IntTensor(labels))
    return ds


In [ ]:
vocalubary, words_matrix, labels=preprocess('hw2/small.txt')

train_ds = encode(vocalubary, words_matrix, labels_dict[train_name])
train_loader = data_utils.DataLoader(train_ds, batch_size=50, shuffle=True)

In [19]:
def preprocess(filename):
    #read in tokens
    vocalubary = set()
    with open(filename) as f:
        words_matrix = []
        lines = f.readlines()
        labels = []
        for l in lines:
            words = l.split()
            labels.append(int(words[0]))
            words_matrix.append(words[1:])
            for w in words[1:]:
                vocalubary.add(w)
    dummy_train = pd.get_dummies(list(vocalubary))
    return dummy_train, vocalubary, words_matrix, labels

def encode(dummy_train, words_matrix, labels):
    bag_of_words_matrix = []
    for lidx, line in enumerate(words_matrix):
        if lidx % 500 == 0:
            print(lidx)
        dummy_test = pd.get_dummies(line)
        dummy_test = dummy_test.reindex(columns = dummy_train.columns, fill_value=0)
        bag_of_words_matrix.append(dummy_test.values.any(axis=0).astype('int'))
    ds = data_utils.TensorDataset(torch.FloatTensor(bag_of_words_matrix), torch.LongTensor(labels))
    return ds


In [77]:
dummy_train, vocalubary, words_matrix, labels=preprocess('hw2/small.txt')

train_ds = encode(dummy_train, words_matrix, labels)
train_loader = data_utils.DataLoader(train_ds, batch_size=50, shuffle=True)

_, _, words_matrix_t, labels_t=preprocess('hw2/small_test.txt')
test_ds = encode(dummy_train, words_matrix_t, labels_t)
test_loader = data_utils.DataLoader(test_ds, batch_size=50, shuffle=True)


0
0


In [84]:
dummy_train

,I,a,an,apple,have,pen
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,1,0,0
3,0,0,1,0,0,0
4,0,0,0,0,0,1
5,0,0,0,0,1,0


In [85]:
for i in test_loader:
    print(i)

[tensor([[0, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 1, 0]], dtype=torch.int32), tensor([0, 1], dtype=torch.int32)]


In [1]:
from sklearn import preprocessing

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import pandas as pd

import time

import math

In [15]:
class model_1(nn.Module):
    # You will implement a simple version of vgg11 (https://arxiv.org/pdf/1409.1556.pdf)
    # Since the shape of image in CIFAR10 is 32x32x3, much smaller than 224x224x3, 
    # the number of channels and hidden units are decreased compared to the architecture in paper
    def __init__(self, vocabulary_size):
        super(model_1, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(vocabulary_size, 2),
            nn.Sigmoid()
        )

    def forward(self, x):
        print(x)
        x = self.fc(x)
        return x


In [4]:
def train(trainloader, net, criterion, optimizer, device):
    for epoch in range(10):  # loop over the dataset multiple times
        start = time.time()
        running_loss = 0.0
        for i, (sentences, labels) in enumerate(trainloader):
            sentences = sentences.to(device)
            labels = labels.to(device)
            #zero the parameter gradients
            optimizer.zero_grad()
            #forward pass
            outputs = net(sentences)
            loss = criterion(outputs, labels)
            #backward pass
            loss.backward()
            #optimize the network
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 2000 mini-batches
                end = time.time()
                print('[epoch %d, iter %5d] loss: %.3f eplased time %.3f' %
                      (epoch + 1, i + 1, running_loss / 100, end-start))
                start = time.time()
                running_loss = 0.0
    print('Finished Training')


def test(testloader, net, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            sentences, labels = data
            sentences = sentences.to(device)
            labels = labels.to(device)
            outputs = net(sentences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))


In [20]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device('cpu')
    train_name = 'hw2/small.txt'
    test_name = 'hw2/small_test.txt'
    dev_name = 'hw2/small_test.txt'
    
    dummy_train, vocalubary, words_matrix, labels=preprocess(train_name)

    train_ds = encode(dummy_train, words_matrix, labels)
    train_loader = data_utils.DataLoader(train_ds, batch_size=50, shuffle=True)

    _, _, words_matrix_test, labels_test = preprocess(test_name)
    test_ds = encode(dummy_train, words_matrix_test, labels_test)
    test_loader = data_utils.DataLoader(test_ds, batch_size=50, shuffle=False)
    
    _, _, words_matrix_dev, labels_dev = preprocess(dev_name)
    dev_ds = encode(dummy_train, words_matrix_dev, labels_dev)
    dev_loader = data_utils.DataLoader(dev_ds, batch_size=50, shuffle=False)
    
    net = model_1(len(vocalubary)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    train(train_loader, net, criterion, optimizer, device)
    test(test_loader, net, device)
    

if __name__== "__main__":
    main()

0
0
0
tensor([[1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.]])
tensor([[0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.],
        [1., 1., 0., 0., 1., 1.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.]])
tensor([[1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.],
        [1., 1., 0., 0., 1., 1.]])
tensor([[1., 0., 1., 1., 1., 0.],
        [1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0.]])
tensor([[1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.],